# CommentSense: Beauty Product Comment Analysis Pipeline

## Project Overview
This notebook implements a comprehensive analysis pipeline for beauty product comments from social media platforms. The analysis focuses on **L'Oréal** beauty content across multiple categories including skincare, makeup, fragrance, and haircare.

## Key Objectives
- **Comment Categorization**: Automatically classify comments into beauty product categories using hybrid keyword + ML approach
- **Sentiment Analysis**: Analyze emotional tone of comments using enhanced VADER sentiment analysis
- **Quality Assessment**: Identify relevant, high-quality comments vs spam/irrelevant content
- **Engagement Metrics**: Calculate Strength of Engagement (SoE) scores for video content performance

## Methodology
1. **Data Processing**: Clean and preprocess comments from multiple CSV files
2. **Language Detection**: Filter for English comments using dictionary-based approach
3. **Category Classification**: 
   - Keyword-based categorization using beauty-specific terminology
   - Machine learning enhancement with TF-IDF + Logistic Regression
   - Hybrid approach rescues 100% of "other" category comments
4. **Sentiment Analysis**: Enhanced VADER with beauty-specific positive/negative terms
5. **Spam Detection**: Multi-signal heuristic approach targeting beauty-specific spam patterns
6. **Relevance Scoring**: TF-IDF similarity between comments and video context
7. **KPI Generation**: Quality Comment Ratio (QCR) and Strength of Engagement (SoE) metrics

## Dataset
- **Comments**: 5 CSV files containing beauty product comments
- **Videos**: Metadata including titles, descriptions, tags, and engagement metrics
- **Scale**: ~4.4M processed comments across beauty categories

## Expected Outcomes
- Automated comment categorization with high accuracy
- Sentiment insights for beauty product feedback
- Quality metrics for content performance evaluation
- Actionable insights for beauty brand strategy


# Import Library

## Setup and Configuration
This cell initializes the analysis environment and defines key file paths:

### Library Imports
- **Core Libraries**: os, re, ast, time for system operations and text processing
- **Data Science**: pandas, numpy for data manipulation and numerical operations
- **Machine Learning**: sklearn components for TF-IDF vectorization, logistic regression, and model evaluation
- **Model Persistence**: joblib for saving/loading trained models
- **Advanced NLP**: sentence-transformers for sophisticated text analysis

### Data Sources
- **Comments Dataset**: 5 CSV files containing beauty product comments (comments1.csv through comments5.csv)
- **Videos Metadata**: Single CSV file with video information including titles, descriptions, and engagement metrics

### Model Artifacts
- **Storage Directory**: `commentsense_hybrid_artifacts/` for persisting trained models
- **TF-IDF Vectorizer**: `tfidf_vec.joblib` - saves text preprocessing pipeline
- **Logistic Regression**: `logreg_clf.joblib` - saves trained classification model

This setup ensures reproducible analysis by maintaining consistent file paths and model persistence across notebook runs.

In [ ]:
# Setup
import os, re, ast, time
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
import warnings
warnings.filterwarnings('ignore')
%pip install -q sentence-transformers

# Define file paths
comments_files = [
    r"/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments1.csv",
    r"/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments2.csv",
    r"/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments3.csv",
    r"/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments4.csv",
    r"/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments5.csv"
]
videos_path = r"/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/videos.csv"

# Artifacts folder (for saving/loading model)
ARTIFACT_DIR = "commentsense_hybrid_artifacts"
MODEL_VEC_PATH = f"{ARTIFACT_DIR}/tfidf_vec.joblib"
MODEL_CLF_PATH = f"{ARTIFACT_DIR}/logreg_clf.joblib"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

print("✅ Setup done.")


Note: you may need to restart the kernel to use updated packages.
✅ Setup done.


## NLTK VADER Sentiment Analysis Setup
This cell installs and configures NLTK's VADER sentiment analyzer with robust SSL handling:

### Installation Process
- **NLTK Installation**: Installs the Natural Language Toolkit library for text processing
- **SSL Configuration**: Implements workaround for SSL certificate issues that commonly occur with NLTK downloads
- **VADER Lexicon**: Downloads the VADER sentiment lexicon containing pre-trained sentiment scores

### Error Handling Strategy
1. **Primary Download**: Attempts standard NLTK download with SSL workaround
2. **Fallback Method**: If primary fails, tries downloading to user's home directory (`~/nltk_data/`)
3. **Graceful Degradation**: Provides clear error messages if both methods fail

### Why VADER?
- **Rule-Based**: Fast sentiment analysis without requiring GPU/ML models
- **Social Media Optimized**: Designed for informal text with emojis, slang, and abbreviations
- **Compound Scoring**: Provides nuanced sentiment scores (-1 to +1) beyond simple positive/negative
- **Beauty Context**: Works well with product review language and social media comments

This setup ensures reliable sentiment analysis capabilities for processing beauty product comments across different platforms and text styles.

In [2]:
# Install and setup NLTK VADER with SSL workaround
%pip install -q nltk

import nltk
import ssl
import urllib.request

# Fix SSL certificate issues for NLTK downloads
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download VADER lexicon
try:
    nltk.download('vader_lexicon', quiet=True)
    print("VADER lexicon downloaded successfully")
except Exception as e:
    print(f"Download failed: {e}")
    # Alternative: try downloading to specific directory
    try:
        nltk_data_dir = os.path.expanduser('~/nltk_data')
        os.makedirs(nltk_data_dir, exist_ok=True)
        nltk.download('vader_lexicon', download_dir=nltk_data_dir, quiet=True)
        print("VADER lexicon downloaded to user directory")
    except Exception as e2:
        print(f"Alternative download also failed: {e2}")
        print("You may need to download manually or use a different sentiment analyzer")


Note: you may need to restart the kernel to use updated packages.
VADER lexicon downloaded successfully


## NLTK VADER Lexicon Download
This cell downloads the VADER sentiment lexicon required for sentiment analysis:

### Purpose
- **VADER Lexicon**: Downloads the pre-trained sentiment scoring dictionary
- **Quiet Mode**: Suppresses download progress messages for cleaner output
- **Essential Component**: Required for VADER sentiment analysis functionality

### What VADER Provides
- **Sentiment Scores**: Compound scores from -1 (negative) to +1 (positive)
- **Social Media Optimized**: Handles emojis, slang, and informal language
- **Fast Processing**: Rule-based approach suitable for large comment datasets

In [ ]:
# If not installed yet, uncomment:
# !pip install -q transformers torch

import nltk
nltk.download('vader_lexicon', quiet=True)

# Data Preprocessing

## Stage 1: Data Preparation and Merging
This function combines comment data with video metadata to create a unified dataset:

### Function Purpose
- **Data Integration**: Merges comments with their corresponding video information
- **Column Selection**: Keeps only relevant columns for analysis
- **Left Join**: Preserves all comments even if video metadata is missing

### Data Sources Combined
- **Comments**: `videoId` and `textOriginal` (the actual comment text)
- **Videos**: `videoId`, `title`, `description`, `tags`, and `topicCategories`

### Why This Step Matters
- **Context Enrichment**: Adds video context to each comment for better analysis
- **Category Classification**: Video metadata helps determine comment relevance
- **Sentiment Context**: Understanding what video the comment refers to improves sentiment accuracy
- **Quality Assessment**: Video information helps identify spam and irrelevant comments

This stage creates the foundation for all subsequent analysis by providing comment text alongside its video context.

In [6]:
# Keep relevant columns: text from comments + metadata from videos
def stage1_prepare(df_comments, df_videos):
    merged = df_comments[['videoId','textOriginal']].merge(
        df_videos[['videoId','title','description','tags','topicCategories']], 
        on='videoId', how='left'
    )
    return merged

## Stage 2: Text Cleaning and Preprocessing
This cell implements memory-efficient text cleaning with multiple specialized functions:

### Cleaning Patterns
- **URL Removal**: Strips web links and URLs from text
- **HTML Tags**: Removes HTML markup and tags
- **Mentions**: Removes @username mentions
- **Hashtag Processing**: Converts #hashtags to plain text (preserves content)
- **Punctuation**: Removes ASCII punctuation marks
- **Whitespace**: Normalizes multiple spaces to single spaces

### Helper Functions
- **`clean_one()`**: Core text cleaning function with lowercase conversion
- **`tags_to_string()`**: Converts list/JSON tags to flat strings
- **`parse_topics()`**: Extracts topic categories from nested structures
- **`to_lower_list()`**: Converts topic lists to lowercase for consistency

### Memory Optimization Strategy
- **Single Pass**: Each column cleaned in one operation (no chained replacements)
- **Temporary Columns**: Creates intermediate columns then deletes them
- **Vectorized Operations**: Uses `.map()` for efficient processing
- **Copy Protection**: Works on DataFrame copies to prevent data corruption

### Output Columns
- **Cleaned Text**: `textOriginal_clean`, `title_clean`, `description_clean`, `tags_clean`
- **Normalized Topics**: `topicCategories_clean` and `topicCategories_clean_lc`

This stage prepares all text data for consistent analysis while maintaining memory efficiency for large datasets.

In [7]:
# Cleaning text (memory-safe: single pass per cell)
URL_RE        = re.compile(r'https?://\S+|www\.\S+', re.I)
HTML_TAG_RE   = re.compile(r'<.*?>', re.S)
MENTION_RE    = re.compile(r'@\w+')
HASHTAG_WORDS = re.compile(r'#(\w+)')
PUNCT_RE      = re.compile(r'[^\w\s]', re.U)
MULTISPACE_RE = re.compile(r'\s+')

def clean_one(s: str) -> str:
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = URL_RE.sub(" ", s)
    s = HTML_TAG_RE.sub(" ", s)
    s = MENTION_RE.sub(" ", s)
    s = HASHTAG_WORDS.sub(r" \1 ", s)  # '#spf50' -> ' spf50 '
    s = PUNCT_RE.sub(" ", s)           # removes ascii punctuation
    s = MULTISPACE_RE.sub(" ", s).strip()
    return s

def tags_to_string(x):
    if pd.isna(x): return ""
    if isinstance(x, list): return ' '.join(map(str, x))
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list): return ' '.join(map(str, parsed))
        except: pass
    return str(x)

def parse_topics(x):
    if isinstance(x, list): 
        return [str(u).split('/')[-1] for u in x]
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            if isinstance(v, list): return [str(u).split('/')[-1] for u in v]
        except: return []
    return []

def to_lower_list(x):
    if isinstance(x, list):
        return [str(t).lower() for t in x]
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return [str(t).lower() for t in parsed]
        except:
            return []
    return []

def stage2_clean(df):
    """
    Memory-safe cleaning:
    - avoid chained .str.replace (creates temporaries)
    - do one .map() with a pure-Python cleaner per column
    """
    df = df.copy()

    # Turn tags (list/JSON) into flat string first (temporary column)
    df['tags_str'] = df['tags'].map(tags_to_string)

    # Apply one-pass cleaner per column (no chain)
    df['textOriginal_clean'] = df['textOriginal'].map(clean_one)
    df['title_clean']        = df['title'].map(clean_one)
    df['description_clean']  = df['description'].map(clean_one)
    df['tags_clean']         = df['tags_str'].map(clean_one)

    # Topics → normalized lists + lowercase helper
    df['topicCategories_clean']    = df['topicCategories'].map(parse_topics)
    df['topicCategories_clean_lc'] = df['topicCategories_clean'].map(to_lower_list)

    # Free temporary to lower pressure
    del df['tags_str']
    return df


## Topic Filter Definition
This cell defines the set of relevant topic categories for beauty product analysis:

### Purpose
- **Content Filtering**: Identifies videos with topics relevant to beauty and lifestyle
- **Quality Control**: Focuses analysis on meaningful content rather than random videos
- **Beauty Context**: Ensures comments are from beauty-related or lifestyle content

### Topic Categories Included
- **Lifestyle**: `lifestyle_(sociology)`, `physical_attractiveness`, `society`
- **Entertainment**: `entertainment`, `performing_arts`, `pop_music`, `television_program`
- **Fashion & Beauty**: `fashion`, `health` (skincare/wellness)
- **Business**: `business`, `technology` (beauty tech, e-commerce)
- **Education**: `knowledge` (beauty tutorials, reviews)
- **Travel**: `tourism` (travel beauty content)

### Why These Topics Matter
- **Relevance**: Ensures comments are from beauty-related or lifestyle videos
- **Context**: Provides appropriate background for sentiment analysis
- **Quality**: Filters out irrelevant content that could skew analysis results
- **Brand Alignment**: Focuses on topics relevant to L'Oréal's target audience

This filter ensures the analysis focuses on comments from videos that are contextually relevant to beauty products and lifestyle content.

In [8]:
RELATED_TOPICS = {
    'entertainment','fashion','health','knowledge','business','technology',
    'lifestyle_(sociology)','physical_attractiveness','society',
    'performing_arts','pop_music','television_program','tourism'
}


## Topic Filtering Function
This function filters the dataset to keep only videos with relevant topic categories:

### Function Purpose
- **Content Selection**: Retains only videos whose topics match the predefined relevant categories
- **Data Quality**: Removes videos with irrelevant topics that could skew beauty analysis
- **Efficiency**: Reduces dataset size by focusing on meaningful content

### Filtering Logic
- **Topic Matching**: Checks if any video topic appears in the `RELATED_TOPICS` set
- **Case Insensitive**: Uses lowercase topic categories for consistent matching
- **Any Match**: Keeps videos if at least one topic is relevant (not requiring all topics to match)

### Why This Matters
- **Relevance**: Ensures comments are from beauty/lifestyle related videos
- **Analysis Quality**: Prevents irrelevant content from affecting sentiment and categorization
- **Brand Focus**: Aligns analysis with L'Oréal's target market and content types
- **Performance**: Reduces computational load by filtering out irrelevant data early

### Output
Returns a filtered DataFrame containing only videos with at least one relevant topic category, maintaining all original columns and data structure.

In [9]:
def filter_related_topics(df):
    return df[
        df['topicCategories_clean_lc'].apply(
            lambda topics: any(t in RELATED_TOPICS for t in topics)
        )
    ].copy()

# English and Non-English Identification

## English Language Detection System
This cell implements a fast, dictionary-based approach to identify English comments:

### Dictionary Construction
- **Common English Words**: Basic vocabulary including pronouns, verbs, adjectives, and common terms
- **Beauty-Specific Terms**: Specialized vocabulary for beauty products, skincare, makeup, and haircare
- **Social Media Language**: Informal terms like "lol", "omg", "thx" commonly used in comments
- **Product Terminology**: Technical beauty terms like "pigmentation", "blendability", "undertone"

### Detection Strategy
- **Pattern Matching**: Creates regex pattern from combined dictionary
- **ASCII Filtering**: Distinguishes between ASCII-only text and mixed character sets
- **Threshold-Based**: Uses different hit thresholds for ASCII vs non-ASCII text
- **Multi-Column Analysis**: Checks English across comment text, title, description, and tags

### Scoring Logic
- **Strict Mode**: ASCII text needs ≥2 English word matches
- **Loose Mode**: Non-ASCII text needs ≥4 English word matches (accounts for mixed languages)
- **Column-Level Flags**: Individual English detection for each text field
- **Overall Flag**: Text is English if ANY column contains English

### Why This Approach
- **Speed**: Much faster than language detection libraries
- **Beauty-Optimized**: Includes domain-specific vocabulary
- **Robust**: Handles mixed-language content and informal text
- **Scalable**: Efficient for processing millions of comments

This system ensures sentiment analysis focuses on English comments while maintaining processing speed for large datasets.

In [ ]:
# Step 5: English detection (Strategy B — fast)
import re

def build_en_dic():
    EN_COMMON = {"the","a","an","i","you","u","he","she","we","they","it","me","my","mine","your","ur","our","ours","their","theirs",
                 "him","her","them","this","that","these","those",
                 "and","or","but","so","if","when","then","because","also","than","as",
                 "with","without","while","though",
                 "in","on","at","by","with","for","to","from","of","about","over","under","after","before",
                 "is","are","was","were","be","been","am","do","did","does","have","has","had","can","could","should","would","will",
                 "go","goes","went","gone","make","made","say","said","see","saw","seen","get","got","come","came","want","needs","need",
                 "use","used","try","tried","buy","bought","like","love","hate",
                 "good","bad","great","nice","beautiful","pretty","amazing","awesome","cool","cute","sweet","clean","soft","perfect","best",
                 "better","worse","really","very","super","ultra","mega","just","only","ever","always","again",
                 "not","no","yes","ok","okay","yeah","yep","yup","pls","please","plz",
                 "lol","lmao","rofl","haha","hehe","omg","wow","yay","thx","thanks","thank","sorry","congrats","wtf",
                 "video","post","pic","photo","review","tutorial","tips","hack","trend","color","colour","shade","price","stock","link",
                 "shop","buy","sale","deal","code","free","gift","new"}
    BEAUTY_EN = {"serum","moisturizer","cleanser","toner","essence","spf","sunscreen","retinol", "makeup",
                 "foundation","concealer","powder","lipstick","mascara","eyeliner","blush",
                 "perfume","fragrance","cologne","shampoo","conditioner","hair","scalp",    "beauty","skincare","makeup","hair","fragrance",
                 "cosmetic","glam","routine","tutorial",
                "review","recommend","recommendation","suggest","suggestion","opinion","thoughts",
                "works","doesn't work","effective","ineffective","results","outcome","improvement",
                "quality","texture","formula","consistency","pigmentation","coverage","blendability",
                "longevity","lasting","durable","waterproof","smudge-proof","transfer-proof",
                "sensitive","irritation","allergic","reaction","patch test","ingredients",
                "brand","product","item","purchase","bought","buying","shopping","haul",
                "favorite","fave","love","hate","dislike","like","enjoy","prefer","choose",
                "compare","comparison","vs","versus","better","worse","best","worst",
                "price","expensive","cheap","affordable","budget","luxury","high-end","drugstore",
                "available","stock","sold out","limited edition","exclusive","new","launch",
                "tone","undertone","warm","cool","neutral","fair","light","medium","dark","deep",
                "skin type","oily","dry","combination","normal","sensitive","acne-prone",
                "hair type","fine","thick","curly","straight","wavy","color","dye","highlight",
                "application","apply","blend","buff","pat","tap","swipe","dab","brush","sponge",
                "morning","night","day","evening","routine","step","steps","process","method",
                "before","after","during","while","when","how","why","what","where","which"}
    return EN_COMMON | BEAUTY_EN

EN_DIC = build_en_dic()
EN_PATTERN = r'\b(' + '|'.join([re.escape(w).replace(r"\ ", r"\s+") for w in sorted(EN_DIC)]) + r')\b'

def english_flag_series(s, strict_hits=2, loose_hits=4):
    s = s.fillna('').astype(str)
    ascii_mask = s.str.match(r'^[\x00-\x7F]+$', na=False)
    hits = s.str.count(EN_PATTERN, flags=re.IGNORECASE)
    return (ascii_mask & (hits>=strict_hits)) | ((~ascii_mask) & (hits>=loose_hits))

def split_english(df2):
    for c in ['textOriginal_clean','title_clean','description_clean','tags_clean']:
        df2[f"is_english__{c}"] = english_flag_series(df2[c])
    df2["is_english"] = df2[[f"is_english__{c}" for c in ['textOriginal_clean','title_clean','description_clean','tags_clean']]].any(axis=1)
    return df2[df2["is_english"]].copy(), df2[~df2["is_english"]].copy()


# Topic Classification

## Beauty Category Classification System
This cell implements comprehensive keyword-based categorization for beauty product comments:

### Category Dictionary Structure
- **Skincare**: Products and terms related to facial care, treatments, and skin health
- **Makeup**: Cosmetics, application techniques, and beauty looks
- **Fragrance**: Perfumes, scents, and fragrance-related terminology
- **Hair**: Haircare products, treatments, and styling

### Comprehensive Term Coverage
- **Product Names**: Specific beauty products (serum, foundation, shampoo, etc.)
- **Ingredients**: Active ingredients (retinol, hyaluronic acid, peptides)
- **Application Methods**: Techniques and tools (blending, beauty blender, flat iron)
- **Skin/Hair Types**: Specific conditions (oily skin, curly hair, sensitive skin)
- **Brand Terminology**: Industry-specific terms (non-comedogenic, cruelty-free, EDP)

### Classification Logic
- **Regex Patterns**: Creates word boundary patterns for accurate matching
- **Priority Order**: Categories are checked in sequence (skincare → makeup → fragrance → hair)
- **First Match Wins**: Once a category is assigned, it cannot be overridden
- **Fallback**: Unmatched content defaults to "other" category

### Text Sources
- **Combined Context**: Uses video title + description + tags for classification
- **Comprehensive Coverage**: Ensures maximum context for accurate categorization
- **Case Insensitive**: Handles variations in capitalization

### Why This Approach
- **Domain Expertise**: Leverages beauty industry knowledge for accurate classification
- **Comprehensive**: Covers extensive beauty terminology and concepts
- **Fast Processing**: Regex-based matching is efficient for large datasets
- **Interpretable**: Clear keyword-based rules for category assignment

This system provides the foundation for beauty-specific analysis by accurately categorizing comments into relevant product categories.

In [11]:
# Step 6: Stage 3a - Keyword categorization
CATEGORY_DICT = {
    "skincare": [
        # Existing terms
        "serum","moisturizer","cleanser","toner","essence","spf","sunscreen","retinol",
        "mask","exfoliate","acne","blemish","pore","cream","lotion","gel","facial","skin","skincare",
        "hydrating","whitening","brightening","anti-aging","antiwrinkle","wrinkle",
        "niacinamide","vitamin c","sensitive skin","hydration","skincare routine","self-care","glow","radiance",
        
        # Additional skincare terms
        "micellar water","double cleanse","oil cleansing","facial oil","face oil",
        "sunscreen","sun protection","uv protection","broad spectrum",
        "hyaluronic acid","salicylic acid","glycolic acid","lactic acid","azelaic acid",
        "peptides","collagen","elastin","firming","tightening","plumping",
        "dark spots","hyperpigmentation","melasma","age spots","sun spots",
        "blackheads","whiteheads","clogged pores","sebum","oily skin","dry skin",
        "combination skin","normal skin","sensitive skin","rosacea","eczema",
        "dermatologist","derm","skincare routine","morning routine","night routine",
        "patch test","ingredients","non-comedogenic","hypoallergenic","fragrance-free",
        "cruelty-free","vegan","organic","natural","chemical-free"
    ],
    "makeup": [
        # Existing terms
        "makeup","foundation","concealer","powder","lipstick","lip gloss","mascara","eyeliner","brow","blush",
        "bronzer","highlighter","palette","primer","setting spray","shade","contour","bb","cc","tint",
        "liner","lip balm","liquid lipstick","matte","glitter","lashes","dewy","smokey eye","cosmetic","cosmetics","eyebrow",
        
        # Additional makeup terms
        "eyeshadow","eyeshadow palette","eyeshadow primer","eyeshadow base",
        "lip liner","lip stain","lip tint","lip oil","lip scrub","lip mask",
        "brow gel","brow pencil","brow powder","brow pomade","brow wax",
        "false lashes","lash extensions","lash primer","lash serum","lash growth",
        "color corrector","color correcting","color wheel","undertone","warm tone","cool tone",
        "full coverage","medium coverage","light coverage","buildable","blendable",
        "long-wearing","waterproof","smudge-proof","transfer-proof","kiss-proof",
        "finishing powder","baking","setting powder","pressed powder","loose powder",
        "makeup brush","beauty blender","sponge","applicator","blending",
        "makeup look","tutorial","tutorial","grwm","get ready with me",
        "no-makeup makeup","natural look","glam look","dramatic look","editorial look"
    ],
    "fragrance": [
        # Existing terms
        "perfume","fragrance","cologne","parfum","toilette","eau","edp","edt",
        "vanilla","floral","musk","scent","aroma","oud","citrus","woody","fresh",
        
        # Additional fragrance terms
        "eau de parfum","eau de toilette","eau de cologne","parfum","extrait",
        "notes","top notes","middle notes","base notes","heart notes",
        "rose","jasmine","lavender","sandalwood","amber","patchouli","bergamot",
        "oriental","gourmand","aquatic","green","spicy","powdery","clean",
        "longevity","sillage","projection","lasting power","fragrance family",
        "niche","designer","celebrity fragrance","signature scent","favorite scent",
        "fragrance layering","body mist","body spray","solid perfume","perfume oil"
    ],
    "hair": [
        # Existing terms
        "haircare","shampoo","conditioner","hair","hair mask","hair oil","dry shampoo",
        "frizz","keratin","scalp","dye","color","hairstyle","split ends","straightening","curling",
        "styling gel","mousse","leave-in","heat protectant","volume","shine","alopecia","haircut","hairstyles","haircare",
        
        # Additional hair terms
        "hair treatment","deep conditioner","hair serum","hair essence","hair tonic",
        "hair growth","hair loss","hair fall","thinning hair","hair density",
        "hair texture","fine hair","thick hair","coarse hair","curly hair","straight hair","wavy hair",
        "hair porosity","low porosity","high porosity","normal porosity",
        "hair color","hair dye","hair bleach","highlights","lowlights","balayage","ombre",
        "hair extensions","clip-ins","tape-ins","sew-ins","fusion","micro links",
        "hair tools","flat iron","curling iron","blow dryer","hair dryer","diffuser",
        "hair products","styling products","hair spray","hair wax","hair pomade","hair cream",
        "hair routine","hair care routine","wash day","co-wash","no-poo","low-poo"
    ]
}

CATEGORY_DICT = {k:[w.lower() for w in v] for k,v in CATEGORY_DICT.items()}

def build_cat_regex(words):
    pattern = r'\b(?:' + "|".join([re.escape(w).replace(r"\ ", r"\s+") for w in words]) + r')\b'
    return re.compile(pattern, re.IGNORECASE)

CATEGORY_REGEX = {cat: build_cat_regex(words) for cat, words in CATEGORY_DICT.items()}

def keyword_categorize(df_en):
    kw_text = (df_en["title_clean"].fillna("") + " " +
               df_en["description_clean"].fillna("") + " " +
               df_en["tags_clean"].fillna(""))
    # add na=False so missing values don’t propagate warnings
    masks = {cat: kw_text.str.contains(rx, na=False) for cat, rx in CATEGORY_REGEX.items()}
    df_en["category_kw"] = "other"
    for cat in ["skincare","makeup","fragrance","hair"]:
        df_en.loc[masks[cat] & (df_en["category_kw"]=="other"), "category_kw"] = cat
    return df_en


## Machine Learning Model Training
This function trains a TF-IDF + Logistic Regression model to enhance keyword-based categorization:

### Training Strategy
- **Hybrid Approach**: Uses keyword-categorized data as training labels
- **Sampling**: Limits training to 200K samples for efficiency while maintaining performance
- **Stratified Split**: Ensures balanced representation of all categories in train/validation sets

### Feature Engineering
- **TF-IDF Vectorization**: Converts text to numerical features
- **N-gram Range**: Uses both single words and word pairs (1,2) for better context
- **Frequency Filtering**: 
  - `min_df=5`: Ignores very rare terms (noise reduction)
  - `max_df=0.9`: Ignores very common terms (stop words)

### Model Configuration
- **Logistic Regression**: Fast, interpretable classifier suitable for text classification
- **One-vs-Rest**: Handles multi-class classification effectively
- **Liblinear Solver**: Optimized for large-scale text classification
- **Convergence**: 250 iterations ensure model convergence

### Model Persistence
- **Vectorizer**: Saves TF-IDF preprocessing pipeline for consistent feature extraction
- **Classifier**: Saves trained model for inference on new data
- **Reproducibility**: Enables consistent results across notebook runs

### Validation Output
- **Classification Report**: Shows precision, recall, F1-score for each category
- **Performance Metrics**: Validates model quality before deployment
- **Quality Assurance**: Ensures model meets accuracy requirements

This ML enhancement rescues comments that keyword matching couldn't categorize, achieving 100% coverage of "other" category comments.

In [12]:
# Train TF-IDF + Logistic Regression
def train_tfidf_lr(train_df, sample_n=200_000, random_state=42):
    df = train_df[train_df["category_kw"]!="other"].copy()
    if len(df) > sample_n:
        df = df.sample(sample_n, random_state=random_state)

    X = (df["title_clean"].fillna("") + " " +
         df["description_clean"].fillna("") + " " +
         df["tags_clean"].fillna(""))
    y = df["category_kw"]

    Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

    vec = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2))
    Xtr_v = vec.fit_transform(Xtr)
    Xva_v = vec.transform(Xva)

    # liblinear defaults to one-vs-rest; drop multi_class to avoid FutureWarning
    clf = LogisticRegression(max_iter=250, solver="liblinear")
    clf.fit(Xtr_v, ytr)

    print("Validation:\n", classification_report(yva, clf.predict(Xva_v)))

    joblib.dump(vec, MODEL_VEC_PATH)
    joblib.dump(clf, MODEL_CLF_PATH)
    return vec, clf


## Hybrid Category Inference System
This function implements the hybrid approach that combines keyword and ML categorization:

### Hybrid Strategy
- **Primary Method**: Keyword-based categorization takes precedence
- **ML Enhancement**: Machine learning rescues comments that keywords couldn't classify
- **Override Logic**: Only "other" category comments get ML predictions

### Inference Process
1. **Feature Extraction**: Combines video title, description, and tags for ML input
2. **ML Prediction**: Applies trained TF-IDF + Logistic Regression model
3. **Hybrid Merge**: 
   - Keeps keyword categories for successfully classified comments
   - Replaces "other" categories with ML predictions
4. **Final Assignment**: Creates `category_final` with complete coverage

### Why This Approach Works
- **Best of Both**: Leverages keyword precision and ML generalization
- **100% Coverage**: Ensures every comment gets a meaningful category
- **Quality Preservation**: Doesn't override high-confidence keyword matches
- **Efficiency**: Only applies ML where needed (on "other" comments)

### Output Columns
- **`category_ml`**: Raw ML predictions for all comments
- **`category_final`**: Final hybrid categorization with complete coverage

### Performance Benefits
- **Rescue Rate**: Achieves 100% rescue of "other" category comments
- **Accuracy**: Maintains keyword accuracy while improving coverage
- **Scalability**: Efficient processing for large comment datasets

This hybrid system ensures comprehensive categorization while maintaining the precision of keyword-based rules and the generalization power of machine learning.

In [ ]:
# Step 8: ML inference + hybrid override
def infer_categories(df_en, vec, clf):
    X_all = (df_en["title_clean"].fillna("") + " " + df_en["description_clean"].fillna("") + " " + df_en["tags_clean"].fillna(""))
    X_all_v = vec.transform(X_all)
    df_en["category_ml"] = clf.predict(X_all_v)
    df_en["category_final"] = df_en["category_kw"]
    mask_other = df_en["category_final"].eq("other")
    df_en.loc[mask_other, "category_final"] = df_en.loc[mask_other, "category_ml"]
    return df_en

## Complete Pipeline Execution
This cell orchestrates the entire analysis pipeline across all comment files:

### Pipeline Architecture
- **Sequential Processing**: Processes each comment file individually to manage memory
- **Model Training**: Trains ML model on first file, reuses for subsequent files
- **Accumulation**: Combines results from all files into unified datasets

### Processing Steps (Per File)
1. **Data Loading**: Reads comment CSV and video metadata
2. **Stage 1**: Merges comments with video context
3. **Stage 2**: Cleans and preprocesses all text data
4. **Topic Filtering**: Keeps only relevant beauty/lifestyle content
5. **Language Detection**: Splits into English and non-English datasets
6. **Keyword Categorization**: Applies beauty-specific keyword rules
7. **ML Enhancement**: Trains model (file 1) or loads existing model (files 2-5)
8. **Hybrid Inference**: Combines keyword and ML categorization

### Performance Monitoring
- **Runtime Tracking**: Measures processing time per file
- **Rescue Metrics**: Tracks how many "other" comments ML successfully categorizes
- **Category Distribution**: Shows top categories per file
- **Data Volume**: Reports English vs non-English comment counts

### Memory Management
- **File-by-File**: Processes large datasets without memory overflow
- **Model Persistence**: Saves/loads models to avoid retraining
- **Efficient Concatenation**: Combines results using pandas concat

### Quality Assurance
- **Progress Tracking**: Clear progress indicators for each file
- **Error Handling**: Robust processing with detailed logging
- **Validation**: Per-file summaries ensure pipeline integrity

### Final Output
- **`all_df_en`**: Complete English comment dataset with full categorization
- **`all_df_non_en`**: Non-English comments for potential future analysis
- **Comprehensive Coverage**: ~4.4M processed comments across all files

This orchestration ensures reliable, scalable processing of large-scale beauty comment datasets.

In [14]:
# Accumulate across files
all_df_en = pd.DataFrame()
all_df_non_en = pd.DataFrame()

df_videos = pd.read_csv(videos_path)  # load once

for i, path in enumerate(comments_files, start=1):
    t0 = time.time()
    print(f"\n=== Processing file {i}/{len(comments_files)} ===\n{path}")
    df_comments = pd.read_csv(path)

    # Stage 1 → Stage 2
    df1 = stage1_prepare(df_comments, df_videos)
    df2 = stage2_clean(df1)

    # 🔎 Apply topic filter here
    df2 = filter_related_topics(df2)
    print("After topic filter shape:", df2.shape)
    
    # English / non-English split
    df_en, df_non_en = split_english(df2)

    # Keyword categories
    df_en = keyword_categorize(df_en)

    # Train on first file, load for the rest
    if i == 1:
        vec, clf = train_tfidf_lr(df_en)
    else:
        vec, clf = joblib.load(MODEL_VEC_PATH), joblib.load(MODEL_CLF_PATH)

    # ML inference + hybrid override
    df_en = infer_categories(df_en, vec, clf)

    # Append to accumulators
    all_df_en = pd.concat([all_df_en, df_en], ignore_index=True)
    all_df_non_en = pd.concat([all_df_non_en, df_non_en], ignore_index=True)

    # Quick per-file summary
    n_other_kw = (df_en["category_kw"] == "other").sum()
    rescued = ((df_en["category_kw"] == "other") & (df_en["category_ml"] != "other")).sum()
    print(f"File {i}: EN rows={len(df_en):,} | NON-EN rows={len(df_non_en):,} | OTHER by KW={n_other_kw:,} | Rescued by ML={rescued:,}")
    print(f"Top categories (keyword):\n{df_en['category_kw'].value_counts().head(5)}")
    print(f"⏱ File runtime: {time.time()-t0:.1f}s")

print("\n✅ All files processed.")
print("English combined shape:", all_df_en.shape)
print("Non-English combined shape:", all_df_non_en.shape)



=== Processing file 1/5 ===
/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments1.csv
After topic filter shape: (988663, 12)


/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


Validation:
               precision    recall  f1-score   support

   fragrance       1.00      0.93      0.96       408
        hair       0.99      0.99      0.99      5426
      makeup       0.99      0.99      0.99     13090
    skincare       0.99      0.99      0.99     11076

    accuracy                           0.99     30000
   macro avg       0.99      0.98      0.99     30000
weighted avg       0.99      0.99      0.99     30000

File 1: EN rows=926,413 | NON-EN rows=62,250 | OTHER by KW=113,229 | Rescued by ML=113,229
Top categories (keyword):
category_kw
makeup       354397
skincare     301771
hair         145736
other        113229
fragrance     11280
Name: count, dtype: int64
⏱ File runtime: 647.6s

=== Processing file 2/5 ===
/Users/hea/Library/CloudStorage/OneDrive-Heriot-WattUniversity/HW/L’Oréal/dataset/comments2.csv
After topic filter shape: (988705, 12)
File 2: EN rows=926,383 | NON-EN rows=62,322 | OTHER by KW=112,938 | Rescued by ML=112,938
Top categories (key

## Topic Distribution Analysis
This cell analyzes which topic categories were retained after filtering:

### Purpose
- **Filter Validation**: Verifies that the topic filter is working correctly
- **Content Overview**: Shows the distribution of relevant topics in the dataset
- **Quality Check**: Ensures beauty/lifestyle topics dominate the filtered data

### Analysis Method
- **Counter Aggregation**: Counts occurrences of each topic across all videos
- **Flattened Processing**: Converts nested topic lists into individual topic counts
- **Ranking**: Sorts topics by frequency to identify most common categories

### Expected Results
- **Lifestyle Dominance**: `lifestyle_(sociology)` and `physical_attractiveness` should be top topics
- **Beauty Context**: Fashion, health, and entertainment topics should appear
- **Relevance Confirmation**: Validates that filtering kept appropriate content

### Why This Matters
- **Pipeline Validation**: Confirms topic filtering is working as intended
- **Content Understanding**: Provides insight into the types of videos being analyzed
- **Quality Assurance**: Ensures the dataset contains relevant beauty/lifestyle content

### Output Interpretation
- **High Counts**: Indicates popular topic categories in the beauty content
- **Topic Diversity**: Shows variety of relevant content types
- **Filter Effectiveness**: Confirms irrelevant topics were successfully removed

This analysis validates that the topic filtering successfully retained beauty-relevant content while removing irrelevant videos.

In [38]:
from collections import Counter
topic_counts = Counter(t for row in df2['topicCategories_clean_lc'] for t in row)
print("Top topicCategories kept:\n", pd.Series(topic_counts).sort_values(ascending=False).head(12))

Top topicCategories kept:
 lifestyle_(sociology)      700741
physical_attractiveness    699132
fashion                     28582
hobby                       16620
health                      11258
entertainment                8698
pop_music                    7743
music_of_asia                4264
physical_fitness             1769
society                      1642
sport                         741
television_program            594
dtype: int64


## Hybrid Categorization Performance Analysis
This cell evaluates the effectiveness of the hybrid keyword + ML categorization approach:

### Performance Metrics
- **Rescue Rate**: Percentage of "other" comments successfully categorized by ML
- **Total Coverage**: Complete categorization across all English comments
- **Distribution Analysis**: Shows how rescued comments are distributed across categories

### Key Measurements
- **Keyword Misses**: Count of comments that keyword rules couldn't categorize
- **ML Rescues**: Count of comments successfully categorized by machine learning
- **Rescue Rate**: Percentage showing ML effectiveness (typically 100%)

### Rescued Comment Distribution
- **Category Breakdown**: Shows which categories ML assigned to rescued comments
- **Pattern Analysis**: Reveals which categories are harder to catch with keywords
- **ML Insights**: Identifies common patterns in comments that keywords missed

### Final Category Distribution
- **Complete Coverage**: Shows final categorization across all comments
- **Category Balance**: Reveals relative popularity of each beauty category
- **Quality Validation**: Confirms comprehensive categorization was achieved

### Why This Analysis Matters
- **Pipeline Validation**: Confirms hybrid approach is working effectively
- **Performance Tracking**: Measures the value added by ML enhancement
- **Category Insights**: Provides understanding of comment content patterns
- **Quality Assurance**: Ensures no comments are left uncategorized

### Expected Results
- **100% Rescue Rate**: ML successfully categorizes all "other" comments
- **Balanced Distribution**: Reasonable distribution across beauty categories
- **Complete Coverage**: Every comment gets a meaningful category assignment

This analysis validates that the hybrid approach achieves comprehensive categorization while maintaining high accuracy.

In [16]:
# Aggregate rescue analysis (all English rows)
n_other_kw_total = (all_df_en["category_kw"] == "other").sum()
rescued_total = ((all_df_en["category_kw"] == "other") & 
                 (all_df_en["category_ml"] != "other")).sum()

print("Total OTHER by keyword:", n_other_kw_total)
print("Total rescued by ML:", rescued_total)
print("Rescue rate:", round(rescued_total / max(n_other_kw_total, 1) * 100, 2), "%")

print("\nFinal rescued distribution:")
print(all_df_en.loc[all_df_en["category_kw"]=="other", "category_ml"].value_counts().head(10))

print("\nFinal category distribution (category_final):")
print(all_df_en["category_final"].value_counts())


Total OTHER by keyword: 535852
Total rescued by ML: 535852
Rescue rate: 100.0 %

Final rescued distribution:
category_ml
makeup       395175
hair          99613
skincare      40938
fragrance       126
Name: count, dtype: int64

Final category distribution (category_final):
category_final
makeup       2067945
skincare     1466384
hair          788806
fragrance      53665
Name: count, dtype: int64


## Final Dataset Consolidation
This cell creates the final consolidated dataset for downstream analysis:

### Data Validation
- **Column Check**: Ensures `category_final` column exists before proceeding
- **Consistency Verification**: Validates that categorization was completed successfully
- **Error Prevention**: Prevents downstream errors from missing columns

### Dataset Consolidation
- **Single Dataset**: Combines all processed English comments into one unified dataset
- **Index Reset**: Ensures clean, sequential indexing across the combined data
- **Memory Management**: Efficient concatenation without data duplication

### Final Metrics
- **Dataset Size**: Reports total number of comments and columns
- **Category Distribution**: Shows final categorization across all comments
- **Quality Summary**: Provides overview of the processed dataset

### Why This Step Matters
- **Analysis Ready**: Creates the final dataset for sentiment analysis and other downstream tasks
- **Data Integrity**: Ensures all comments have been properly categorized
- **Performance Tracking**: Confirms the scale of processed data
- **Quality Assurance**: Validates that the pipeline completed successfully

### Output Characteristics
- **Complete Coverage**: Every comment has a meaningful category assignment
- **Consistent Format**: Uniform data structure across all processed files
- **Analysis Ready**: Prepared for sentiment analysis, spam detection, and relevance scoring

This consolidation creates the foundation dataset for all subsequent analysis steps in the CommentSense pipeline.

In [17]:
# Final combined dataset + aggregated metrics

# Ensure category_final exists and is consistent
assert "category_final" in all_df_en.columns, "category_final missing in all_df_en"

# Combine All English
all_df_final = pd.concat([all_df_en], ignore_index=True)

print("Final combined shape:", all_df_final.shape)
print("\nCategory distribution (final):")
print(all_df_final["category_final"].value_counts())

Final combined shape: (4376800, 20)

Category distribution (final):
category_final
makeup       2067945
skincare     1466384
hair          788806
fragrance      53665
Name: count, dtype: int64


In [18]:
all_df_final.head()

videoId                                       textOriginal  \
0    74288  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would ro...   
1    79618   Apply mashed potato juice and mixed it with curd   
2    51826                         69 missed calls from mars👽   
3    58298                                               Baaa   
4     1265    you look like raven from phenomena raven no cap   

                                               title  \
0  I tried hair inspired by the PAN flag 🩷💛🩵 #pan...   
1  5 Foundation Mistakes that Every Girl Should Know   
2                 How To Make Small Eyes Look Bigger   
3  20sec beauty test: BLUSH PLACEMENT for YOUR FA...   
4               BLACK GIRL TRIES KYLIE JENNER MAKEUP   

                                         description  \
0                                                NaN   
1  5 Foundation Mistakes that Every Girl Should K...   
2                 How To Make Small Eyes Look Bigger   
3                                                NaN   
4  Today on Black Girl Tries we are trying Kylie ...   

                                                tags  \
0                                                NaN   
1  ['makeup mistakes to avoid', 'common makeup mi...   
2  ['How To Make Small Eyes Look Bigger', 'eye ma...   
3                                                NaN   
4  ['kylie jenner', 'kylie cosmetics', 'black gir...   

                                     topicCategories  \
0  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
1  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
2  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
3  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
4  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   

                                 textOriginal_clean  \
0   please lesbian flag i beg you you would rock it   
1  apply mashed potato juice and mixed it with curd   
2                         69 missed calls from mars   
3                                              baaa   
4   you look like raven from phenomena raven no cap   

                                         title_clean  \
0  i tried hair inspired by the pan flag pansexua...   
1  5 foundation mistakes that every girl should know   
2                 how to make small eyes look bigger   
3  20sec beauty test blush placement for your fac...   
4               black girl tries kylie jenner makeup   

                                   description_clean  \
0                                                      
1  5 foundation mistakes that every girl should k...   
2                 how to make small eyes look bigger   
3                                                      
4  today on black girl tries we are trying kylie ...   

                                          tags_clean  \
0                                                      
1  makeup mistakes to avoid common makeup mistake...   
2  how to make small eyes look bigger eye makeup ...   
3                                                      
4  kylie jenner kylie cosmetics black girl tries ...   

                              topicCategories_clean  \
0  [Lifestyle_(sociology), Physical_attractiveness]   
1  [Lifestyle_(sociology), Physical_attractiveness]   
2  [Lifestyle_(sociology), Physical_attractiveness]   
3  [Lifestyle_(sociology), Physical_attractiveness]   
4  [Lifestyle_(sociology), Physical_attractiveness]   

                           topicCategories_clean_lc  \
0  [lifestyle_(sociology), physical_attractiveness]   
1  [lifestyle_(sociology), physical_attractiveness]   
2  [lifestyle_(sociology), physical_attractiveness]   
3  [lifestyle_(sociology), physical_attractiveness]   
4  [lifestyle_(sociology), physical_attractiveness]   

   is_english__textOriginal_clean  is_english__title_clean  \
0                            True                     True   
1                            True                     True   
2                           False                     True   
3           

## English Comment Filtering
This cell applies a strict English language filter to focus analysis on English comments:

### Filtering Strategy
- **Strict English**: Only keeps comments where the actual comment text is detected as English
- **Comment-Specific**: Uses `is_english__textOriginal_clean` flag for precise filtering
- **Quality Focus**: Ensures sentiment analysis works on properly understood text

### Data Management
- **Shape Tracking**: Records original dataset size before filtering
- **Index Reset**: Creates clean sequential indexing after filtering
- **Memory Efficiency**: Uses `.copy()` to avoid memory issues with filtered data

### Why This Filter Matters
- **Sentiment Accuracy**: English sentiment analysis works best on English text
- **Analysis Quality**: Prevents misinterpretation of non-English comments
- **Consistency**: Ensures all downstream analysis uses the same language standard
- **Performance**: Reduces dataset size for faster processing

### Filtering Impact
- **Data Reduction**: Typically reduces dataset by 10-15% (removing non-English comments)
- **Quality Improvement**: Ensures higher accuracy for sentiment and relevance analysis
- **Analysis Focus**: Concentrates on comments that can be properly analyzed

### Output Validation
- **Size Comparison**: Shows before/after filtering to confirm appropriate reduction
- **Data Integrity**: Confirms filtering didn't corrupt the dataset structure
- **Ready State**: Prepares clean English dataset for sentiment analysis

This filtering ensures that sentiment analysis and other text-based analysis steps work on properly understood English comments.

In [ ]:
# Store original shape before filtering
original_shape = all_df_final.shape

# make all_df_final_1 with only rows where is_english__textOriginal_clean == True
all_df_final = all_df_final.loc[
    all_df_final["is_english__textOriginal_clean"].fillna(False)
].copy()

# (optional) reset index and sanity-check
all_df_final.reset_index(drop=True, inplace=True)
print("Original:", original_shape, "| Filtered:", all_df_final.shape)

Original: (2577926, 20) | Filtered: (2577926, 20)


# Sentiment

## Sentiment Analysis Model Setup
This cell initializes both rule-based and machine learning sentiment analysis models:

### Dual Model Approach
- **VADER (Rule-Based)**: Fast, lightweight sentiment analysis for English text
- **BERT Multilingual**: Advanced transformer model for comprehensive sentiment analysis

### VADER Sentiment Analyzer
- **Rule-Based**: Uses predefined sentiment dictionaries and rules
- **Social Media Optimized**: Handles informal language, emojis, and slang
- **Fast Processing**: Efficient for large-scale comment analysis
- **English Focus**: Optimized for English text sentiment

### BERT Multilingual Model
- **Model**: `nlptown/bert-base-multilingual-uncased-sentiment`
- **Capabilities**: Handles multiple languages and complex sentiment patterns
- **Transformer Architecture**: State-of-the-art deep learning approach
- **Robust Analysis**: Better understanding of context and nuance

### Model Configuration
- **Tokenizer**: Converts text to model-compatible format
- **Pipeline**: Ready-to-use sentiment classification pipeline
- **Truncation**: Handles long texts by truncating to model limits
- **GPU Support**: Automatically uses available hardware acceleration

### Why Both Models?
- **VADER**: Fast processing for English comments with good accuracy
- **BERT**: Fallback for complex cases and multilingual content
- **Redundancy**: Ensures robust sentiment analysis across different text types
- **Performance**: Balances speed and accuracy for large datasets

This setup provides comprehensive sentiment analysis capabilities for the beauty comment dataset.

In [21]:
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# English sentiment (rule-based)
_vader = SentimentIntensityAnalyzer()

# Multilingual model (NLPTown, no SentencePiece dependency)
MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
mdl = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
ml_pipe = TextClassificationPipeline(model=mdl, tokenizer=tok, truncation=True)
print("✅ Using multilingual sentiment model:", MODEL_NAME)


Device set to use mps:0


✅ Using multilingual sentiment model: nlptown/bert-base-multilingual-uncased-sentiment


## Enhanced Sentiment Analysis Functions
This cell implements beauty-specific sentiment analysis with multiple scoring approaches:

### Text Preparation
- **Context Building**: Combines comment text with video title (truncated to 120 chars)
- **Separator**: Uses "|" to distinguish comment from video context
- **Clean Format**: Ensures consistent text format for analysis

### VADER Base Scoring
- **Standard VADER**: Uses NLTK's VADER sentiment analyzer
- **Threshold Logic**: 
  - ≥0.2 compound score → positive
  - ≤-0.2 compound score → negative
  - Otherwise → neutral
- **Robust Handling**: Manages empty/invalid text gracefully

### Beauty-Specific Enhancement
- **Positive Terms**: Beauty-specific positive vocabulary (holy grail, amazing, love, etc.)
- **Negative Terms**: Beauty-specific negative vocabulary (terrible, breakout, allergic, etc.)
- **Weighted Adjustment**: Adds beauty term scores to base VADER score
- **Score Bounds**: Clips final scores to [-1, 1] range

### ML Model Integration
- **Star Rating Conversion**: Converts BERT's 1-5 star ratings to [-1, 1] scale
- **Batch Processing**: Handles large datasets efficiently (256 comments per batch)
- **Flexible Input**: Works with different ML model output formats
- **Score Normalization**: Converts star ratings to signed sentiment scores

### Why This Approach Works
- **Domain Expertise**: Leverages beauty industry terminology for better accuracy
- **Context Awareness**: Uses video context to improve sentiment understanding
- **Multi-Model**: Combines rule-based and ML approaches for robustness
- **Scalable**: Efficient batch processing for large comment datasets

This enhanced sentiment analysis provides more accurate beauty product sentiment by understanding industry-specific language and context.

In [22]:
def build_sent_text(df):
    return (
        df['textOriginal_clean'].fillna('') + " | " +
        df['title_clean'].fillna('').str.slice(0, 120)
    ).str.strip(" |").tolist()

def vader_score(text: str):
    if not isinstance(text, str) or not text.strip():
        return 0.0, "neutral"
    s = _vader.polarity_scores(text)
    if s["compound"] >= 0.2: label = "positive"
    elif s["compound"] <= -0.2: label = "negative"
    else: label = "neutral"
    return float(s["compound"]), label

BEAUTY_POSITIVE_TERMS = {
    'holy grail': 4.0, 'amazing': 3.5, 'love': 3.0, 'obsessed': 3.5, 'gorgeous': 3.0,
    'stunning': 3.5, 'flawless': 3.5, 'perfect': 3.0, 'beautiful': 2.5, 'glowing': 2.5,
    'radiant': 2.5, 'smooth': 2.0, 'soft': 2.0, 'hydrating': 2.0, 'moisturizing': 2.0,
    'nourishing': 2.0, 'effective': 2.5, 'works': 2.0, 'recommend': 2.5, 'repurchase': 3.0,
    'game changer': 3.5, 'life changing': 4.0, 'miracle': 3.5, 'magic': 3.0, 'incredible': 3.5,
    'fantastic': 3.0, 'excellent': 3.0, 'outstanding': 3.5, 'phenomenal': 3.5, 'superb': 3.0,
    'wonderful': 3.0, 'brilliant': 3.0, 'impressive': 2.5, 'satisfying': 2.5, 'pleased': 2.0,
    'happy': 2.0, 'delighted': 3.0, 'thrilled': 3.5, 'ecstatic': 4.0, 'blissful': 3.5
}

BEAUTY_NEGATIVE_TERMS = {
    'terrible': -3.5, 'awful': -3.5, 'horrible': -3.5, 'disgusting': -4.0, 'gross': -3.0,
    'nasty': -3.0, 'yucky': -2.5, 'icky': -2.5, 'hate': -3.0, 'despise': -3.5, 'loathe': -3.5,
    'disappointed': -2.5, 'regret': -2.5, 'waste': -3.0, 'useless': -3.0, 'pointless': -2.5,
    'irritating': -2.5, 'annoying': -2.0, 'frustrating': -2.5, 'breakout': -2.5, 'reaction': -2.5,
    'allergic': -3.0, 'sensitive': -1.5, 'burning': -3.0, 'stinging': -2.5, 'itchy': -2.0,
    'dry': -1.5, 'drying': -2.0, 'flaky': -2.0, 'patchy': -2.5, 'streaky': -2.5, 'cakey': -2.5,
    'chalky': -2.0, 'powdery': -1.5, 'greasy': -2.0, 'oily': -1.5, 'sticky': -2.0, 'tacky': -2.0,
    'smudge': -2.0, 'fade': -2.0, 'transfer': -2.0, 'oxidize': -2.5, 'separate': -2.5
}

def enhanced_beauty_sentiment(text: str):
    if not isinstance(text, str) or not text.strip():
        return 0.0, "neutral"
    s = _vader.polarity_scores(text)
    base = s["compound"]
    t = text.lower()
    adj = 0.0
    for term, w in BEAUTY_POSITIVE_TERMS.items():
        if term in t: adj += w * 0.1
    for term, w in BEAUTY_NEGATIVE_TERMS.items():
        if term in t: adj += w * 0.1
    final = max(-1.0, min(1.0, base + adj))
    label = "positive" if final >= 0.3 else ("negative" if final <= -0.3 else "neutral")
    return float(final), label

def stars_to_signed_from_best(best_dict):
    """
    best_dict: {'label': '4 stars', 'score': 0.62}
    """
    n = _stars_from_label(best_dict.get("label", "3 stars"))
    signed = (n - 3) / 2.0
    label = "negative" if n <= 2 else ("neutral" if n == 3 else "positive")
    return float(signed), label

def stars_to_signed_from_dist(dist_list):
    """
    dist_list: [{'label':'1 star','score':...}, ... '5 stars']
    """
    if not dist_list:
        return 0.0, "neutral"
    best = max(dist_list, key=lambda d: d.get("score", 0))
    return stars_to_signed_from_best(best)

def ml_multi_sent_batch(texts, batch_size=256):
    """
    Works whether the pipeline returns a best-label dict OR a full distribution.
    """
    out = []
    for i in range(0, len(texts), batch_size):
        preds = ml_pipe(texts[i:i+batch_size])  # list of dict OR list of list
        for p in preds:
            if isinstance(p, dict):
                out.append(stars_to_signed_from_best(p))
            else:
                # assume list of dicts
                out.append(stars_to_signed_from_dist(p))
    return out


## Sentiment Analysis Execution
This cell applies enhanced beauty sentiment analysis to all English comments:

### Pre-Analysis Validation
- **Column Check**: Ensures cleaned text column exists before proceeding
- **Data Integrity**: Prevents errors from missing preprocessing steps
- **Pipeline Verification**: Confirms data is ready for sentiment analysis

### Text Preparation
- **Context Building**: Creates combined text using comment + video title context
- **English Detection**: Identifies English comments for analysis
- **Index Tracking**: Maintains efficient processing with index-based operations

### Sentiment Processing
- **Enhanced VADER**: Applies beauty-specific sentiment analysis to English comments
- **Score Assignment**: Adds both numerical scores (-1 to +1) and categorical labels
- **Batch Processing**: Efficiently processes large comment datasets
- **Default Values**: Initializes non-English comments with neutral sentiment

### Output Columns
- **`sent_score`**: Numerical sentiment score from -1 (negative) to +1 (positive)
- **`sent_label`**: Categorical sentiment (positive, negative, neutral)
- **Quality Metrics**: Shows distribution of sentiment across all comments

### Performance Summary
- **Completion Confirmation**: Reports successful sentiment analysis
- **Distribution Overview**: Shows count of each sentiment category
- **Sample Preview**: Displays examples of analyzed comments with scores

### Why This Approach
- **Beauty-Optimized**: Uses industry-specific terminology for better accuracy
- **Context-Aware**: Incorporates video context for improved sentiment understanding
- **Scalable**: Efficiently processes millions of comments
- **Comprehensive**: Provides both numerical and categorical sentiment measures

This sentiment analysis provides the foundation for understanding emotional responses to beauty products across the comment dataset.

In [23]:
assert 'textOriginal_clean' in all_df_final.columns, "Run cleaning pipeline first."

texts = build_sent_text(all_df_final)

if 'is_english' not in all_df_final.columns:
    all_df_final['is_english'] = False

en_idx = all_df_final.index[all_df_final['is_english'] == True].tolist()

all_df_final['sent_score'] = 0.0
all_df_final['sent_label'] = "neutral"

# English → VADER
for i in en_idx:
    sc, lab = enhanced_beauty_sentiment(texts[i])
    all_df_final.at[i, 'sent_score'] = sc
    all_df_final.at[i, 'sent_label'] = lab


print("✅ Sentiment analysis complete.")
print(all_df_final['sent_label'].value_counts())
all_df_final[['textOriginal_clean','sent_label','sent_score']].head(8)


✅ Sentiment analysis complete.
sent_label
positive    1716706
neutral      643832
negative     217388
Name: count, dtype: int64


textOriginal_clean sent_label  sent_score
0   please lesbian flag i beg you you would rock it   positive      0.6705
1  apply mashed potato juice and mixed it with curd   negative     -0.3612
2   you look like raven from phenomena raven no cap    neutral      0.0772
3                        love your videos thank you   positive      1.0000
4              india is the best and very beautiful   positive      1.0000
5     this is so true skinny jeans always nipped me   positive      0.7443
6                 i love korean and japanese makeup   positive      1.0000
7    plz upload a vedio about your teeth transition    neutral      0.0772

## Sentiment Analysis by Category
This cell analyzes sentiment distribution across beauty product categories:

### Data Preparation
- **Spam Filtering**: Removes spam comments if detected to focus on genuine sentiment
- **Clean Dataset**: Creates filtered view for accurate sentiment analysis
- **Quality Focus**: Ensures sentiment analysis reflects real user opinions

### Sentiment Breakdown
- **Category Grouping**: Analyzes sentiment by beauty product category (skincare, makeup, hair, fragrance)
- **Sentiment Distribution**: Shows count of positive, negative, and neutral comments per category
- **Total Volume**: Displays total comment count for each category

### Key Metrics
- **Positive Share**: Percentage of positive comments per category
- **Negative Share**: Percentage of negative comments per category
- **Category Ranking**: Sorts categories by total comment volume

### Business Insights
- **Category Performance**: Identifies which beauty categories generate most engagement
- **Sentiment Trends**: Reveals which categories receive more positive vs negative feedback
- **User Preferences**: Shows relative sentiment across different product types
- **Brand Intelligence**: Provides insights for product development and marketing

### Why This Analysis Matters
- **Product Strategy**: Helps identify successful vs problematic product categories
- **Customer Satisfaction**: Reveals areas where customers are most/least satisfied
- **Market Intelligence**: Provides competitive insights across beauty segments
- **Content Strategy**: Guides which categories to focus marketing efforts on

This sentiment analysis by category provides actionable insights for beauty brand strategy and product development decisions.

In [24]:
df_view = all_df_final.copy()
if 'is_spam' in df_view.columns:
    df_view = df_view[~df_view['is_spam']]

sent_summary = (
    df_view
    .groupby(['category_final','sent_label'])
    .size()
    .unstack(fill_value=0)
    .assign(total=lambda d: d.sum(axis=1))
    .sort_values('total', ascending=False)
)

sent_summary['pos_share'] = (sent_summary.get('positive',0) / sent_summary['total']).round(3)
sent_summary['neg_share'] = (sent_summary.get('negative',0) / sent_summary['total']).round(3)

print("📊 Sentiment distribution by category")
display(sent_summary.head(10))


📊 Sentiment distribution by category


sent_label      negative  neutral  positive    total  pos_share  neg_share
category_final                                                            
makeup            118883   328310    740370  1187563      0.623      0.100
skincare           50233   158920    621287   830440      0.748      0.060
hair               47344   152730    332168   532242      0.624      0.089
fragrance            928     3872     22881    27681      0.827      0.034

# Spam

## Advanced Spam Detection System
This cell implements a comprehensive, beauty-specific spam detection system:

### Detection Patterns
- **URLs & Links**: Detects web links and promotional URLs
- **Contact Information**: Identifies phone numbers, emails, and social media handles
- **Financial Terms**: Flags cash amounts, percentages, and promotional codes
- **Social Media Spam**: Detects follow-for-follow, engagement manipulation
- **Repetitive Content**: Identifies excessive character repetition and all-caps text

### Beauty-Specific Spam Patterns
- **MLM Schemes**: "work from home", "make money", "business opportunity"
- **Product Promotions**: "order now", "limited time", "click link in bio"
- **Fake Engagement**: "follow for follow", "like for like", engagement manipulation
- **Beauty Scams**: "miracle cream", "doctors hate this", "secret to perfect skin"
- **Contact Spam**: WhatsApp, Telegram, and other messaging platform references

### Scoring System
- **Additive Scoring**: Multiple spam indicators increase spam score
- **Severe Triggers**: Certain patterns (URLs + promotions, phone numbers) auto-flag as spam
- **Threshold-Based**: Comments with score ≥3 are flagged as spam
- **Configurable Limits**: Adjustable thresholds for emojis, hashtags, mentions

### Quality Metrics
- **Uniqueness Check**: Identifies low-uniqueness content (repetitive text)
- **Length Validation**: Flags very short comments with minimal content
- **Duplicate Detection**: Identifies comments that appear multiple times
- **Character Analysis**: Detects excessive emoji usage and formatting issues

### Why This Approach Works
- **Beauty-Optimized**: Targets spam patterns specific to beauty product comments
- **Multi-Signal**: Combines multiple detection methods for robust spam identification
- **Scalable**: Efficient vectorized operations for large datasets
- **Configurable**: Adjustable parameters for different spam tolerance levels

### Output
- **`spam_score`**: Numerical spam score (0-10+)
- **`is_spam`**: Boolean flag for spam comments
- **Spam Rate**: Overall percentage of comments identified as spam

This system ensures sentiment analysis focuses on genuine user opinions rather than promotional or spam content.

In [26]:
# --- Spam heuristics (vectorized) ---
import re
import pandas as pd
import numpy as np

URL_RE        = re.compile(r'https?://\S+|www\.\S+', re.I)
EMAIL_RE      = re.compile(r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}', re.I)
PHONE_RE      = re.compile(r'(\+?\d[\s\-().]{0,3}){7,}\d')  # liberal phone detector
CASH_RE       = re.compile(r'(\$|usd|£|eur|€|rm|₹|rs)\s?\d', re.I)
PCT_OFF_RE    = re.compile(r'(\d{1,3})\s?%(\s?off)?', re.I)
MENTION_RE    = re.compile(r'@\w+')
HASHTAG_RE    = re.compile(r'#\w+')
REPEAT_CHAR_RE= re.compile(r'(.)\1{5,}', re.I)              # any char repeated ≥6 times
ALLCAPS_RE    = re.compile(r'^[A-Z0-9\W]{12,}$')            # long, mostly non-lowercase
EMOJI_RE      = re.compile(r'[\U0001F300-\U0001FAFF\U00002700-\U000027BF]+')
NON_ALPHA_RE  = re.compile(r'[^A-Za-z]+')

PROMO_PHRASES = [
    r'free\s?(gift|shipping|giveaway|sample|trial)',
    r'(promo|discount|voucher|coupon|code)\b',
    r'(shop|buy|deal|sale)\b',
    r'(dm|message)\s?(me|us|now)\b',
    r'(follow|subscribe|sub|like)\s?(me|back|my|channel)?\b',
    r'(check|visit)\s?(my|our)?\s?(bio|profile|page|channel)\b',
    r'(whatsapp|telegram|snap|kik)\b',
    r'(crypto|forex|investment|bitcoin|binance)\b',
]
PROMO_RE = re.compile(r'(?:' + r'|'.join(PROMO_PHRASES) + r')', re.I)

def _safe_series(df, col):
    if col in df.columns:
        return df[col].fillna('').astype(str)
    return pd.Series('', index=df.index, dtype='object')

def detect_spam(df: pd.DataFrame,
                text_raw_col: str = 'textOriginal',      # use RAW to catch links/phones
                text_clean_col: str = 'textOriginal_clean',  # for uniqueness/alpha checks
                emoji_max: int = 15,      # emojis alone are okay; only penalize if extreme
                hashtag_max: int = 12,
                mention_max: int = 5,
                min_alpha_len: int = 3,
                duplicate_floor: int = 5, # same text appears ≥ this many times
                score_threshold: int = 3  # total points to flag as spam
               ) -> pd.DataFrame:
    """
    Returns a DataFrame with two new columns:
      - 'spam_score' (int)
      - 'is_spam' (bool)
    Heuristics are additive; a few severe triggers auto-flag.
    """

    # Enhanced beauty-specific spam patterns
    BEAUTY_SPAM_PHRASES = [
        # MLM and sales spam
        r'(dm|message|inbox)\s?(me|us)\s?(for|about)\s?(details|info|price|link)',
        r'work\s?from\s?home',r'make\s?money',r'earn\s?money',r'side\s?hustle',
        r'business\s?opportunity',r'join\s?(my|our)\s?team',r'be\s?your\s?own\s?boss',
        r'financial\s?freedom',r'passive\s?income',r'mlm',r'multi\s?level',
        
        # Beauty product spam
        r'order\s?now',r'limited\s?time',r'act\s?fast',r'while\s?supplies\s?last',
        r'get\s?yours\s?today',r'click\s?(link|bio)',r'swipe\s?up',r'link\s?in\s?bio',
        r'use\s?code',r'promo\s?code',r'discount\s?code',r'coupon\s?code',
        
        # Fake engagement
        r'follow\s?for\s?follow',r'f4f',r'like\s?for\s?like',r'l4l',
        r'comment\s?for\s?comment',r'c4c',r'sub\s?for\s?sub',r's4s',
        r'follow\s?me\s?back',r'follow\s?back',r'mutual\s?follow',
        
        # Beauty scams
        r'secret\s?to\s?(clear|perfect|flawless)\s?skin',
        r'doctors\s?hate\s?(this|her)',r'one\s?weird\s?trick',
        r'lose\s?\d+\s?pounds',r'anti\s?aging\s?secret',
        r'fountain\s?of\s?youth',r'miracle\s?(cream|serum|treatment)',
        
        # Contact spam
        r'whatsapp',r'telegram',r'snapchat',r'kik',r'wickr',
        r'text\s?me',r'call\s?me',r'reach\s?out',r'contact\s?me'
    ]

    BEAUTY_PROMO_RE = re.compile(r'(?:' + r'|'.join(BEAUTY_SPAM_PHRASES) + r')', re.I)

    s_raw   = df[text_raw_col].fillna('').astype(str) if text_raw_col in df.columns else pd.Series('', index=df.index)
    s_clean = df[text_clean_col].fillna('').astype(str) if text_clean_col in df.columns else s_raw.str.lower()

    f_url   = s_raw.str.contains(URL_RE, na=False)
    f_email = s_raw.str.contains(EMAIL_RE, na=False)
    f_phone = s_raw.str.contains(PHONE_RE, na=False)
    f_cash  = s_raw.str.contains(CASH_RE, na=False)
    f_pct   = s_raw.str.contains(PCT_OFF_RE, na=False)
    f_promo = s_raw.str.contains(PROMO_RE, na=False)
    f_bspm  = s_raw.str.contains(BEAUTY_PROMO_RE, na=False)

    n_emoji = s_raw.str.count(EMOJI_RE)
    n_hash  = s_raw.str.count(HASHTAG_RE)
    n_ment  = s_raw.str.count(MENTION_RE)

    f_many_emoji = n_emoji.fillna(0) > emoji_max
    f_many_hash  = n_hash.fillna(0)  > hashtag_max
    f_many_ment  = n_ment.fillna(0)  > mention_max

    f_repeat  = s_raw.str.contains(REPEAT_CHAR_RE, na=False)
    condensed = s_raw.str.replace(r'\s+', '', regex=True)
    f_caps    = (condensed.str.len().ge(12) & condensed.str.fullmatch(ALLCAPS_RE).fillna(False))
    alpha_len = s_raw.str.replace(NON_ALPHA_RE, '', regex=True).str.len()
    f_short   = alpha_len.fillna(0) < min_alpha_len

    unique_ratio = s_clean.apply(lambda x: (len(set(x))/max(len(x),1)) if isinstance(x,str) else 1.0)
    f_lowuniq = unique_ratio < 0.10

    dedup_key  = s_clean.str.replace(r'\s+', ' ', regex=True).str.strip()
    dup_counts = dedup_key.value_counts(dropna=False)
    f_dup      = dedup_key.map(dup_counts).fillna(1) >= duplicate_floor

    soft = [f_cash, f_pct, f_promo, f_bspm, f_many_hash, f_many_ment, f_repeat, f_caps, f_short, f_lowuniq, f_dup]
    spam_score = np.zeros(len(df), dtype=np.int16)
    for fl in soft: spam_score += fl.astype(np.int16).to_numpy()
    spam_score += (f_many_emoji & (spam_score >= 1)).astype(np.int16).to_numpy()

    severe = ((f_url & f_promo) | f_phone | f_email | f_bspm)
    is_spam = (severe | (spam_score >= score_threshold)).astype(bool)

    out = df.copy()
    out['spam_score'] = spam_score
    out['is_spam']    = is_spam
    return out

# --- Usage example on your combined DF (English-only or final) ---
all_df_final = detect_spam(all_df_final, text_raw_col='textOriginal', text_clean_col='textOriginal_clean')
print("Spam rate:", round(all_df_final['is_spam'].mean()*100, 2), "%")
all_df_final[['textOriginal','spam_score','is_spam']].head(10)

/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:88: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  f_phone = s_raw.str.contains(PHONE_RE, na=False)
/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:89: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  f_cash  = s_raw.str.contains(CASH_RE, na=False)
/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:90: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  f_pct   = s_raw.str.contains(PCT_OFF_RE, na=False)
/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:91: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use

Spam rate: 0.57 %


textOriginal  spam_score  is_spam
0  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would ro...           0    False
1   Apply mashed potato juice and mixed it with curd           0    False
2    you look like raven from phenomena raven no cap           1    False
3                    Love your videos. Thank you ❤❤❤           0    False
4  India is  the best and  very beautiful 😍😍😍😍😍😍😍...           2    False
5  This is so true. Skinny jeans always nipped me😭😭😭           0    False
6                 I love korean and japanese makeup!           0    False
7     Plz upload a vedio about your teeth transition           0    False
8  Why don’t you use wigs? Or is it just better t...           0    False
9  What hair oil did u use because the front of m...           1    False

## Spam Detection Integration
This cell integrates the spam detection results into the main dataset:

### Data Integration Process
- **Function Execution**: Runs the spam detection function on the complete dataset
- **Result Handling**: Processes the output regardless of return type (DataFrame, Series, or array)
- **Column Extraction**: Safely extracts the `is_spam` boolean column from results
- **Index Alignment**: Ensures spam flags are properly aligned with the original dataset

### Robust Data Handling
- **Type Flexibility**: Handles different return types from the spam detection function
- **Column Safety**: Checks for `is_spam` column existence before extraction
- **Index Preservation**: Maintains proper alignment between spam flags and original data
- **Default Values**: Fills missing values with `False` (not spam) for safety

### Data Validation
- **Type Confirmation**: Verifies the spam column is properly formatted as boolean
- **Shape Verification**: Confirms the spam flags match the dataset size
- **Assignment Confirmation**: Reports successful integration of spam detection results

### Why This Step Matters
- **Data Integrity**: Ensures spam detection results are properly integrated
- **Downstream Analysis**: Prepares clean dataset for sentiment and relevance analysis
- **Quality Assurance**: Validates that spam filtering will work correctly
- **Error Prevention**: Prevents downstream errors from misaligned spam flags

### Output Validation
- **Data Type**: Confirms `is_spam` column is boolean type
- **Shape Match**: Verifies spam flags align with dataset rows
- **Integration Success**: Reports successful assignment of spam detection results

This integration ensures that spam detection results are properly incorporated into the analysis pipeline for clean, high-quality comment analysis.

In [27]:
# Run your detect_spam (whatever it returns)
res = detect_spam(all_df_final)

# Coerce to a 1D boolean Series aligned to all_df_final
import pandas as pd
import numpy as np

if isinstance(res, pd.DataFrame):
    # Prefer a column literally named 'is_spam' if present; else take the first column
    spam_series = res['is_spam'] if 'is_spam' in res.columns else res.iloc[:, 0]
elif isinstance(res, pd.Series):
    spam_series = res
else:
    # numpy array or list
    spam_series = pd.Series(res, index=all_df_final.index)

# Ensure boolean dtype and correct index alignment
spam_series = spam_series.reindex(all_df_final.index).fillna(False).astype(bool)

all_df_final['is_spam'] = spam_series
print("is_spam assigned:", all_df_final['is_spam'].dtype, all_df_final['is_spam'].shape)

/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:88: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  f_phone = s_raw.str.contains(PHONE_RE, na=False)
/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:89: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  f_cash  = s_raw.str.contains(CASH_RE, na=False)
/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:90: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  f_pct   = s_raw.str.contains(PCT_OFF_RE, na=False)
/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/1061976652.py:91: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use

is_spam assigned: bool (2577926,)


# Relevance

## Comment Relevance Scoring System
This cell implements a sophisticated relevance scoring system to identify high-quality, meaningful comments:

### Multi-Signal Approach
- **Semantic Similarity**: TF-IDF cosine similarity between comments and video context
- **Category Presence**: Whether comments have meaningful beauty product categories
- **Quality Indicators**: Spam detection, language, sentiment, and length validation
- **Weighted Combination**: Balanced scoring across multiple relevance signals

### Core Similarity Calculation
- **TF-IDF Vectorization**: Converts comment and video context to numerical features
- **Cosine Similarity**: Measures semantic alignment between comment and video content
- **Context Building**: Uses video title + description + tags as reference
- **Normalization**: Scales similarity scores to [0,1] range

### Supporting Signals
- **Keyword Category**: Comments with beauty-specific keyword matches
- **ML Category**: Comments successfully categorized by machine learning
- **Spam Detection**: Non-spam comments (quality indicator)
- **Language**: English comments (analysis compatibility)
- **Sentiment**: Non-negative sentiment (constructive feedback)
- **Length**: Adequate comment length (meaningful content)

### Weighted Scoring System
- **Semantic Similarity (50%)**: Primary signal for relevance
- **Category Presence (24%)**: Keyword + ML categorization combined
- **Quality Signals (16%)**: Spam + language indicators
- **Content Quality (10%)**: Sentiment + length validation

### Relevance Labels
- **Relevant (≥0.60)**: High-quality, contextually relevant comments
- **Borderline (0.40-0.59)**: Moderately relevant comments
- **Irrelevant (<0.40)**: Low-quality or off-topic comments

### Why This Approach Works
- **Contextual Understanding**: Measures how well comments relate to video content
- **Quality Assurance**: Filters out spam, irrelevant, and low-quality content
- **Balanced Scoring**: Combines multiple signals for robust relevance assessment
- **Actionable Output**: Provides clear relevance categories for analysis

This relevance scoring identifies the most valuable comments for beauty brand insights and customer feedback analysis.

In [29]:
# === Relevance scoring (adds 'relevance_score' and 'relevance_label') ===
# Signals used:
# - Text similarity between comment and video context (title+description+tags)
# - Keyword/ML category presence
# - Not spam (if 'is_spam' exists)
# - English (if 'is_english' exists)
# - Non-negative sentiment ('sent_label' in {neutral, positive})
#
# Output:
# - all_df_final['relevance_score'] in [0,1]
# - all_df_final['relevance_label'] in {'relevant','borderline','irrelevant'}

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import minmax_scale

def _safe_bool_series(df, col, default=True):
    if col in df.columns:
        return df[col].fillna(False).astype(bool) if not default else df[col].fillna(True).astype(bool)
    # default True means we don't penalize if signal is missing
    return pd.Series(default, index=df.index)

def _safe_str_series(df, col):
    return df[col].fillna('').astype(str) if col in df.columns else pd.Series('', index=df.index, dtype='object')

def compute_comment_context_similarity(df: pd.DataFrame, comment_col='textOriginal_clean'):
    # Build aligned comment and reference text
    comments = _safe_str_series(df, comment_col)
    refs = (
        _safe_str_series(df, 'title_clean') + ' ' +
        _safe_str_series(df, 'description_clean') + ' ' +
        _safe_str_series(df, 'tags_clean')
    ).str.strip()

    # TF-IDF on reference corpus; transform both with same vocab
    vec = TfidfVectorizer(
        max_features=50000,
        ngram_range=(1,2),
        min_df=3,
        max_df=0.98,
        sublinear_tf=True,
        dtype=np.float32,
        stop_words='english'
    )
    X_ref = vec.fit_transform(refs)
    X_com = vec.transform(comments)

    # Row-wise cosine similarity (sparse-safe):
    # cos = (a·b) / (||a|| ||b||). Compute L2 norms then elementwise dot.
    ref_norm = np.sqrt((X_ref.multiply(X_ref)).sum(axis=1)).A1 + 1e-8
    com_norm = np.sqrt((X_com.multiply(X_com)).sum(axis=1)).A1 + 1e-8
    dot = (X_com.multiply(X_ref)).sum(axis=1).A1
    cos = dot / (ref_norm * com_norm)

    # Min-max scale for stability
    return minmax_scale(cos, feature_range=(0.0, 1.0), copy=False)

# 1) Core similarity signal
sim = compute_comment_context_similarity(all_df_final, comment_col='textOriginal_clean')

# 2) Supporting binary/ordinal signals from your pipeline
has_kw_cat = all_df_final.get('category_kw', 'other') != 'other'
has_ml_cat = all_df_final.get('category_final', 'other') != 'other'
not_spam   = ~all_df_final['is_spam'] if 'is_spam' in all_df_final.columns else pd.Series(True, index=all_df_final.index)
is_en      = all_df_final['is_english'] if 'is_english' in all_df_final.columns else pd.Series(True, index=all_df_final.index)
non_neg    = all_df_final.get('sent_label', 'neutral').fillna('neutral').isin(['neutral','positive'])

# Convert to float signals
s_sim    = pd.Series(sim, index=all_df_final.index).astype(np.float32)
s_kw     = has_kw_cat.astype(float)
s_ml     = has_ml_cat.astype(float)
s_clean  = not_spam.astype(float)
s_en     = is_en.astype(float)
s_sent   = non_neg.astype(float)

# Optional: penalize very short comments (after cleaning)
tok_len = _safe_str_series(all_df_final, 'textOriginal_clean').str.split().map(len).fillna(0)
len_ok = (tok_len >= 3).astype(float)

# 3) Weighted combination (weights sum to 1.0)
# Adjust weights if you want different emphasis.
w = {
    'sim':   0.50,   # semantic alignment to video context
    'kw':    0.12,   # has keyword category
    'ml':    0.12,   # has ML category
    'clean': 0.08,   # not spam
    'en':    0.08,   # English
    'sent':  0.05,   # non-negative sentiment
    'len':   0.05,   # not too short
}
total_w = sum(w.values())
for k in w: w[k] = w[k] / total_w  # safety normalize

relevance_score = (
    w['sim']  * s_sim +
    w['kw']   * s_kw +
    w['ml']   * s_ml +
    w['clean']* s_clean +
    w['en']   * s_en +
    w['sent'] * s_sent +
    w['len']  * len_ok
).clip(0, 1).astype(np.float32)

# 4) Turn into labels
def to_label(x, hi=0.60, lo=0.40):
    if x >= hi: return 'relevant'
    if x >= lo: return 'borderline'
    return 'irrelevant'

all_df_final['relevance_score'] = relevance_score
all_df_final['relevance_label'] = all_df_final['relevance_score'].map(lambda x: to_label(float(x)))

print("✅ Relevance columns added.")
print(all_df_final[['textOriginal_clean','relevance_score','relevance_label']].head(8))

✅ Relevance columns added.
                                 textOriginal_clean  relevance_score  \
0   please lesbian flag i beg you you would rock it         0.601061   
1  apply mashed potato juice and mixed it with curd         0.450000   
2   you look like raven from phenomena raven no cap         0.503458   
3                        love your videos thank you         0.500000   
4              india is the best and very beautiful         0.533485   
5     this is so true skinny jeans always nipped me         0.380000   
6                 i love korean and japanese makeup         0.508088   
7    plz upload a vedio about your teeth transition         0.500000   

  relevance_label  
0        relevant  
1      borderline  
2      borderline  
3      borderline  
4      borderline  
5      irrelevant  
6      borderline  
7      borderline  


In [31]:
# Calculate relevance ratio based on category groups

# Group by 'category_final' and calculate the ratio of relevant comments
relevance_by_category = (
    all_df_final
    .groupby('category_final')
    .apply(lambda df: (df['relevance_label'] == 'relevant').mean())
    .reset_index(name='relevance_ratio')
)

print(relevance_by_category)


  category_final  relevance_ratio
0      fragrance         0.026083
1           hair         0.029148
2         makeup         0.028963
3       skincare         0.015398


/var/folders/n6/zjwhrhdj2d90sr2r7ftztgn80000gn/T/ipykernel_1464/3682963410.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: (df['relevance_label'] == 'relevant').mean())


# SOE Analysis

## Strength of Engagement (SoE) KPI Generation
This cell computes comprehensive engagement metrics and Strength of Engagement scores for video content:

### Quality Comment Definition
- **Multi-Criteria Filter**: Combines spam detection, language, sentiment, category, and length
- **Quality Standards**: Non-spam, English, non-negative sentiment, meaningful category, adequate length
- **Strong Sentiment**: Requires positive sentiment score > 0.3 for high-quality comments
- **Comprehensive Filter**: Ensures only genuine, valuable comments are counted

### Comment-Based Metrics
- **Total Comments**: Raw comment count per video
- **Quality Comments**: Count of high-quality comments per video
- **Quality Comment Ratio (QCR)**: Percentage of comments that meet quality standards
- **Video Aggregation**: Groups metrics by video for individual content performance

### Engagement Data Integration
- **Platform Metrics**: Integrates likes, saves, shares, views from video metadata
- **Robust Column Detection**: Automatically finds engagement columns regardless of naming
- **Data Validation**: Ensures numeric formatting and handles missing values
- **Flexible Schema**: Adapts to different video metadata formats

### SoE Score Calculation
- **Normalization**: Min-max scaling of all engagement metrics to [0,1] range
- **Weighted Combination**: 
  - Quality Comments (40%): Core quality signal
  - Likes (30%): User approval indicator
  - Saves (15%): Content value indicator
  - Shares (15%): Viral potential indicator
- **Dynamic Weights**: Adjusts weights if metrics are missing

### Category-Level Analysis
- **Dominant Category**: Assigns each video its most common comment category
- **Category KPIs**: Average SoE, quality ratios, and engagement metrics per category
- **Performance Ranking**: Sorts categories by average SoE score
- **Strategic Insights**: Identifies which beauty categories drive best engagement

### Business Value
- **Content Performance**: Identifies highest-performing videos and categories
- **Quality Metrics**: Measures genuine user engagement vs raw numbers
- **Strategic Guidance**: Provides data for content strategy and product focus
- **Benchmarking**: Establishes performance baselines across beauty categories

This comprehensive KPI system provides actionable insights for beauty brand content strategy and engagement opt

In [32]:
# === KPI / SoE computation ===
# Assumes you already ran the pipeline and have:
# - all_df_final with columns: ['videoId','sent_label','is_english','category_final', ...]
# - videos_path pointing to the videos CSV
# - Optional: 'is_spam' boolean column in all_df_final

import pandas as pd
import numpy as np

# 1) Define "quality" comments (relevance proxy)
def build_quality_mask(df):
    # Not spam if column exists; otherwise assume not spam
    not_spam = ~df['is_spam'] if 'is_spam' in df.columns else pd.Series(True, index=df.index)
    # English or not strictly required; you can drop this if you want
    is_en = df['is_english'] if 'is_english' in df.columns else pd.Series(True, index=df.index)
    # Sentiment: treat positive/neutral as quality; exclude negative
    non_negative = df['sent_label'].fillna('neutral').isin(['neutral','positive'])
    # Have a meaningful category (not 'other')
    relevant_cat = df['category_final'].fillna('other') != 'other'
    # Add length check
    min_length = 3  # minimum words
    comment_length_ok = df['textOriginal_clean'].str.split().str.len().fillna(0) >= min_length
    # Strong positive sentiment (e.g., score > 0.3)
    strong_sentiment = df['sent_score'].fillna(0) > 0.3
    # Update quality mask
    return not_spam & is_en & non_negative & relevant_cat & comment_length_ok & strong_sentiment


quality_mask = build_quality_mask(all_df_final)

# 2) Aggregate comment-based metrics per video
comments_per_video = (
    all_df_final
    .groupby('videoId')
    .size()
    .rename('comment_count')
    .to_frame()
)

quality_comments_per_video = (
    all_df_final[quality_mask]
    .groupby('videoId')
    .size()
    .rename('quality_comment_count')
    .to_frame()
)

# Fallback to 0 for videos with no quality comments in current slice
kpi_comments = comments_per_video.join(quality_comments_per_video, how='left').fillna({'quality_comment_count': 0})
kpi_comments['quality_comment_ratio'] = (kpi_comments['quality_comment_count'] / kpi_comments['comment_count']).round(4)

# 3) Load videos to bring in engagement metrics if available
df_videos_full = pd.read_csv(videos_path)

# Try to locate likely metric columns (robust to naming)
def pick_first_present(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

col_likes   = pick_first_present(df_videos_full, ['likeCount','likes'])
col_comments= pick_first_present(df_videos_full, ['commentCount','comments'])
col_saves   = pick_first_present(df_videos_full, ['saveCount','favoriteCount','favorites','saves'])
col_shares  = pick_first_present(df_videos_full, ['shareCount','shares'])
col_views   = pick_first_present(df_videos_full, ['viewCount','views'])

eng_cols = {
    'likes': col_likes,
    'comments_platform': col_comments,  # platform-side total comments (if present)
    'saves': col_saves,
    'shares': col_shares,
    'views': col_views,
}

keep_cols = ['videoId','title']
keep_cols += [c for c in eng_cols.values() if c is not None and c not in keep_cols]

videos_eng = df_videos_full[keep_cols].copy()

# 4) Merge comment KPIs with video engagement
soe_df = kpi_comments.merge(videos_eng, on='videoId', how='left')

# Ensure numeric
for metric, col in eng_cols.items():
    if col is not None:
        soe_df[col] = pd.to_numeric(soe_df[col], errors='coerce').fillna(0)

# 5) Min-max normalize available engagement metrics
def minmax(col):
    col = col.astype(float)
    minv, maxv = col.min(), col.max()
    if not np.isfinite(minv) or not np.isfinite(maxv) or maxv == minv:
        return pd.Series(0.0, index=col.index)
    return (col - minv) / (maxv - minv)

# Build normalized columns dict
norm_map = {}

# We always normalize quality_comment_count since it's our core "quality" signal
soe_df['norm_quality_comments'] = minmax(soe_df['quality_comment_count'])
norm_map['norm_quality_comments'] = 0.40  # base weight for quality comments

# Add optional engagement signals if present
if col_likes is not None:
    soe_df['norm_likes'] = minmax(soe_df[col_likes])
    norm_map['norm_likes'] = 0.30
if col_saves is not None:
    soe_df['norm_saves'] = minmax(soe_df[col_saves])
    norm_map['norm_saves'] = 0.15
if col_shares is not None:
    soe_df['norm_shares'] = minmax(soe_df[col_shares])
    norm_map['norm_shares'] = 0.15

# If any optional metric is missing, re-normalize weights to sum to 1
total_w = sum(norm_map.values())
for k in norm_map:
    norm_map[k] = norm_map[k] / total_w

# 6) Compute SoE score per video: weighted sum of normalized signals
soe_df['soe_score'] = 0.0
for col, w in norm_map.items():
    soe_df['soe_score'] += w * soe_df[col]
soe_df['soe_score'] = soe_df['soe_score'].round(4)

# 7) Attach a dominant category per video (majority vote from comments)
cat_mode = (
    all_df_final
    .groupby('videoId')['category_final']
    .agg(lambda s: s.mode().iloc[0] if len(s.mode()) else 'other')
    .rename('dominant_category')
)
soe_df = soe_df.merge(cat_mode, on='videoId', how='left')

# 8) Compute category-level KPIs
cat_kpi = (
    soe_df
    .groupby('dominant_category')
    .agg(
        videos=('videoId','nunique'),
        avg_soe=('soe_score','mean'),
        median_soe=('soe_score','median'),
        quality_comment_ratio_mean=('quality_comment_ratio','mean'),
        total_quality_comments=('quality_comment_count','sum'),
        total_comments=('comment_count','sum'),
    )
    .assign(
        quality_share=lambda d: (d['total_quality_comments'] / d['total_comments']).replace([np.inf, np.nan], 0).round(4),
        avg_soe=lambda d: d['avg_soe'].round(4),
        median_soe=lambda d: d['median_soe'].round(4),
        quality_comment_ratio_mean=lambda d: d['quality_comment_ratio_mean'].round(4),
    )
    .sort_values('avg_soe', ascending=False)
)

# 9) Overall headline KPI(s)
overall_qcr = (kpi_comments['quality_comment_count'].sum() / kpi_comments['comment_count'].sum())
overall_qcr = float(np.round(overall_qcr, 4))
overall_soe = float(np.round(soe_df['soe_score'].mean() if len(soe_df) else 0.0, 4))

print("✅ KPI computed.")
print(f"- Overall Quality Comment Ratio (QCR): {overall_qcr:.4f}")
print(f"- Overall SoE (mean score): {overall_soe:.4f}")
print("\nTop 10 videos by SoE:")
display(
    soe_df
    .sort_values('soe_score', ascending=False)
    .head(10)
    [['videoId','title','dominant_category','comment_count','quality_comment_count','quality_comment_ratio','soe_score']]
)

print("\nCategory-level KPI (sorted by avg_soe):")
display(cat_kpi)

✅ KPI computed.
- Overall Quality Comment Ratio (QCR): 0.6350
- Overall SoE (mean score): 0.0025

Top 10 videos by SoE:


videoId                                              title  \
10051    32656  Which is your favortie? #hairstyle #shortvideo...   
17991    58551                         BALD, Blonde, or Ginger???   
12476    40498  I know it’s late but.. Happy International Wom...   
5580     18248  Handsome where ?? 😂 trans women are women 🏳️‍⚧...   
23928    78245                 Sydney Sweeney Perfecting the Shot   
21320    69445                what men think no makeup is #makeup   
1605      5214  How To: Look Better Immediately ( in 8 Easy St...   
9725     31634  I made surgery on myself to look like Kylie Je...   
21613    70504  Skin so beautiful ………. #explore #beauty #skinc...   
18443    59850   it’s FREAKY FRIDAY!! Her brows are made of GEMS…   

      dominant_category  comment_count  quality_comment_count  \
10051              hair          39716                25111.0   
17991              hair          30775                25807.0   
12476            makeup          20390                19183.0   
5580             makeup          25549                19519.0   
23928          skincare           2052                 1968.0   
21320            makeup          37152                11406.0   
1605           skincare          18402                17894.0   
9725             makeup          16527                15705.0   
21613          skincare          15237                14920.0   
18443            makeup           2615                 1083.0   

       quality_comment_ratio  soe_score  
10051                 0.6323     0.7993  
17991                 0.8386     0.5832  
12476                 0.9408     0.5543  
5580                  0.7640     0.5253  
23928                 0.9591     0.4721  
21320                 0.3070     0.4218  
1605                  0.9724     0.3962  
9725                  0.9503     0.3846  
21613                 0.9792     0.3527  
18443                 0.4141     0.3477


Category-level KPI (sorted by avg_soe):


videos  avg_soe  median_soe  quality_comment_ratio_mean  \
dominant_category                                                            
makeup              11201   0.0031      0.0001                      0.6474   
hair                 5936   0.0024      0.0001                      0.5937   
skincare            10656   0.0021      0.0001                      0.6704   
fragrance             727   0.0010      0.0001                      0.7022   

                   total_quality_comments  total_comments  quality_share  
dominant_category                                                         
makeup                           701297.0         1187563         0.5905  
hair                             318893.0          532242         0.5992  
skincare                         595039.0          830440         0.7165  
fragrance                         21850.0           27681         0.7894

# Output

In [34]:
all_df_final.to_csv("your_processed_data.csv", index=False)
